## PDS Group 7

In [1]:
import pandas as pd
import numpy as np

pd.set_option("mode.copy_on_write", True)

In [ ]:
# Load the processed ARCOs data and display its info
arcos = pd.read_parquet("arcos_processed.parquet")
arcos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162771 entries, 0 to 162770
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   BUYER_STATE   162771 non-null  object 
 1   BUYER_COUNTY  162771 non-null  object 
 2   YEAR          162771 non-null  int32  
 3   TOTAL_MME     162771 non-null  float64
dtypes: float64(1), int32(1), object(2)
memory usage: 4.3+ MB


In [ ]:
# check for missing values in each column
arcos.isna().sum()

BUYER_STATE     0
BUYER_COUNTY    0
YEAR            0
TOTAL_MME       0
dtype: int64